In [1]:
using Bloqade
using Random
using PythonCall
using Optim
plt = pyimport("matplotlib.pyplot");

In [9]:
function loss_piecewise_linear(atoms::AtomList, x::AbstractVector{T}) where {T}
    @assert length(x) == 5
    Rb=sqrt(3*sqrt(10))*4
    C6 = 2π*862690
    Ω_max = 2π*2.5
    Δ_end = C6 / Rb^6
    Δ_start = -2π*10
    Δ0 = Δ_end
    T_max = 0.8

    # the strength of the detunings at each step takes the optimizing x as their input
    Δs = piecewise_linear(clocks = T[0.0, 0.05, 0.2, 0.3, 0.4, 0.5, 0.6, 0.75, T_max],
            values = T[Δ_start, Δ_start, Δ0*x[1], Δ0*x[2], Δ0*x[3], Δ0*x[4], Δ0*x[5], Δ_end, Δ_end])
    Ωs = piecewise_linear(clocks = T[0.0, 0.05, 0.1, 0.2, 0.6, 0.7, 0.75, T_max], values = T[0, 0, 0, Ω_max, Ω_max, 0, 0, 0])

    hamiltonian = rydberg_h(atoms; Ω = Ωs, Δ = Δs)
    subspace = blockade_subspace(atoms, Rb)
    prob = SchrodingerProblem(zero_state(Complex{T}, subspace), T_max, hamiltonian)
    emulate!(prob)
    return -rydberg_density_sum(prob.reg), prob.reg, Ωs, Δs
end

loss_piecewise_linear (generic function with 1 method)

In [10]:
function generateOpt(seed::Int64)
    Random.seed!(seed)
    atoms = generate_sites(SquareLattice(), 5,5; scale = 4) |> random_dropout(0.2)

    x0 = [-6.0, -3.0, 1.0, 3.0, 6.0]; # initial point for the optimization
    optresult = Optim.optimize(x -> loss_piecewise_linear(atoms, x)[1], x0)
    return optresult.minimizer
end

generateOpt (generic function with 1 method)

In [12]:
optim = generateOpt(1)
for i in 2:20
    optimInst = generateOpt(i)
    for j in 1:length(optim)
        optim[j] += optimInst[j]
        optim[j] /= 2
    end
end
optim

5-element Vector{Float64}:
 -34.357805747236654
   1.4329770373209492
   7.5693393460213
   9.973535516555557
  19.50077146687081